In [29]:
import os
import json
import csv

def count_characters_in_string(input_string):
    letter_count = 0
    for char in input_string:
        if char.isalpha():
            letter_count += 1

    return letter_count

def extract_values(directory, person):
    audience, example = os.path.split(directory)[1].split("_", 1)
    
    file_path = str(directory)+"/snapshot_final_assistant_with_scores.json"
    #print(file_path)
    
    with open(file_path, 'r') as file:
        data = json.load(file)
        
        extracted_values = []
        
        contents = []
        rouge_scores = []
        bleu_scores = []
        bleurt_scores = []
        ter_scores = []
        
        
        for obj in data:
            #print(obj)
            for keys, values in obj.items():
                #print(keys, values)
                
                # Getting all the occurences of values
                if keys == 'content':
                    contents.append(values)
                if keys == 'rougeL':
                    rouge_scores.append(values)
                if keys == 'bleu':
                    bleu_scores.append(values)
                if keys == 'scores':
                    bleurt_scores.append(values[0])
                if keys == 'score':
                    ter_scores.append(values)
        
        #print(contents)
        if len(rouge_scores) == 1: # To manage the None_none category adding 0,0 such that it stays formatted as (manual,base,add)
            rouge_scores.append(0)
            rouge_scores.append(0)
            
            bleu_scores.append(0)
            bleu_scores.append(0)
            
            bleurt_scores.append(0)
            bleurt_scores.append(0)
            
            ter_scores.append(0)
            ter_scores.append(0)
        
        
        final_summary = contents[-1].replace("\n","")
        word_count = len(contents[-1].split(" "))
        character_count = count_characters_in_string(contents[-1])
        
        #print(final_summary)
        #print("word count =",word_count)
        #print("character_count =",character_count)
        #print("rouge_scores =",rouge_scores)
        #print("bleu_scores =",bleu_scores)
        #print("bleurt_scores =",bleurt_scores)
        #print("ter_scores =",ter_scores)
                
        #extracted_values = [person, final_summary, word_count, character_count, rouge_scores, bleu_scores, bleurt_scores, ter_scores]
        
        # Flattening out the values
        extracted_values = [
            [audience, example, person, final_summary, word_count, character_count, "manual", rouge_scores[0], bleu_scores[0], bleurt_scores[0], ter_scores[0]],
            [audience, example, person, final_summary, word_count, character_count, "baseline", rouge_scores[1], bleu_scores[1], bleurt_scores[1], ter_scores[1]],
            [audience, example, person, final_summary, word_count, character_count, "additional", rouge_scores[2], bleu_scores[2], bleurt_scores[2], ter_scores[2]]
        ]
        
        return extracted_values

        
def process_persons_directories(directories, persons):
    output_rows =[]
    for person in persons:
        for directory in directories:
            extracted_values = extract_values(directory, person)
            for ground_truths in extracted_values:
                output_rows.append(ground_truths)
    return output_rows


#def write_to_csv(file_path, rows):
    #headers = ['person', 'final_summary', 'word_count', 'character_count', 'ground_truth', 'rouge', 'bleu', 'bleurt', 'ter']
    #with open(file_path, 'w', newline='') as file:
        #writer = csv.writer(file)
        #writer.writerow(headers)
        #writer.writerows(rows)
    
    #print("CSV file created successfully.")

In [30]:
persons = [1, 2]
directories = [
                "../prompt_engineering/p"+str(person)+"/none_none", "../prompt_engineering/p"+str(person)+"/none_manual_example", "../prompt_engineering/p"+str(person)+"/none_masked_manual_example", "../prompt_engineering/p"+str(person)+"/none_masked_template",
                "../prompt_engineering/p"+str(person)+"/self_none", "../prompt_engineering/p"+str(person)+"/self_manual_example", "../prompt_engineering/p"+str(person)+"/self_masked_manual_example", "../prompt_engineering/p"+str(person)+"/self_masked_template",
                "../prompt_engineering/p"+str(person)+"/peer_none", "../prompt_engineering/p"+str(person)+"/peer_manual_example", "../prompt_engineering/p"+str(person)+"/peer_masked_manual_example", "../prompt_engineering/p"+str(person)+"/peer_masked_template",
                "../prompt_engineering/p"+str(person)+"/manager_none", "../prompt_engineering/p"+str(person)+"/manager_manual_example", "../prompt_engineering/p"+str(person)+"/manager_masked_manual_example", "../prompt_engineering/p"+str(person)+"/manager_masked_template"
              ]     
    
output_rows = process_persons_directories(directories, persons)

In [31]:
len(output_rows)

96

In [34]:
import pandas as pd
import numpy as np
from itertools import chain

file_path = '../prompt_engineering/data.csv'
column_headers = ['audience', 'example', 'person', 'final_summary', 'word_count', 'character_count', 'ground_truth', 'rouge', 'bleu', 'bleurt', 'ter']
# The extra additions were to accomodate the flattening of the values based on ground truths
row_headers = [
                ["none_none"]*3, ["none_manual"]*3, ["none_masked_manual"]*3, ["none_masked_template"]*3,
                ["self_none"]*3, ["self_manual"]*3, ["self_masked_manual"]*3, ["self_masked_template"]*3,
                ["peer_none"]*3, ["peer_manual"]*3, ["peer_masked_manual"]*3, ["peer_masked_template"]*3,
                ["manager_none"]*3, ["manager_manual"]*3, ["manager_masked_manual"]*3, ["manager_masked_template"]*3
              ]  *len(persons)
row_headers_flatted = list(chain.from_iterable(row_headers))

df = pd.DataFrame(output_rows, columns = column_headers)

In [35]:
df

,audience,example,person,final_summary,word_count,character_count,ground_truth,rouge,bleu,bleurt,ter
0,none,none,1,The user investigated an event in the intellig...,288,1555,manual,0.120930,0.000000,-0.880093,92.013889
1,none,none,1,The user investigated an event in the intellig...,288,1555,baseline,0.000000,0.000000,0.000000,0.000000
2,none,none,1,The user investigated an event in the intellig...,288,1555,additional,0.000000,0.000000,0.000000,0.000000
3,none,manual_example,1,This user conducted an investigation into even...,190,1077,manual,0.347826,0.199549,-0.295862,80.000000
4,none,manual_example,1,This user conducted an investigation into even...,190,1077,baseline,0.364754,0.269663,-0.265608,114.736842
...,...,...,...,...,...,...,...,...,...,...,...
91,manager,masked_manual_example,2,The user investigated a range of topics relate...,171,1061,baseline,0.219873,0.128963,-0.203020,143.859649
92,manager,masked_manual_example,2,The user investigated a range of topics relate...,171,1061,additional,0.251889,0.150839,-0.142269,104.093567
93,manager,masked_template,2,They focused on 9 main topics in this analysis...,294,1611,manual,0.128736,0.000000,-0.903173,91.554054
94,manager,masked_template,2,They focused on 9 main topics in this analysis...,294,1611,baseline,0.279534,0.333986,-0.078632,91.216216


In [38]:
df.to_csv(file_path, index=False)